In [107]:
import pandas as pd
import time
import ccxt
import os
import numpy as np
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST 
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
import random
import krakenex
from pykrakenapi import KrakenAPI
from datetime import datetime
import questionary

import functions

In [6]:
load_dotenv()

ALPACA_API_KEY_ID = os.getenv('ALPACA_API_KEY_ID')
ALPACA_API_SECRET_KEY = os.getenv('ALPACA_API_SECRET_KEY')

KRAKEN_API_KEY_ID =  os.getenv('KRAKEN_API_KEY_ID')
KRAKEN_API_SECRET_KEY = os.getenv('KRAKEN_API_SECRET_KEY')

In [99]:
#Kraken Direct Api

kraken_api = krakenex.API(KRAKEN_API_KEY_ID, KRAKEN_API_SECRET_KEY)

#print(kraken_api.query_public("AssetPairs")["result"])

crypto_tickers = ['BTC/USD','ETH/USD','LUNA/USD','SOL/USD','XRP/USD','ADA/USD',
    'AVAX/USD','DOT/USD','DOGE/USD','SHIB/USD','MATIC/USD',
    'DAI/USD','LTC/USD','ATOM/USD','LINK/USD','UNI/USD','TRX/USD','BCH/USD']

k = KrakenAPI(kraken_api)

all_ohlc = pd.DataFrame()

dataframes = {}

for ticker in crypto_tickers:
    ticker_ohlc, last = k.get_ohlc_data(ticker.replace('/', ''),1440,1646672400)
    ticker_ohlc['symbol'] = ticker
    dataframes[ticker] = ticker_ohlc.drop(['open', 'high', 'low', 'volume', 'vwap', 'count', 'time'], axis=1).reset_index()
    time.sleep(1)

#print(dataframes['LTC/USD'])


all_ohlc = pd.concat(dataframes.values(), join="inner")

print(all_ohlc)



        dtime     close   symbol
0  2022-04-09  42412.20  BTC/USD
1  2022-04-08  42282.60  BTC/USD
2  2022-04-07  43469.80  BTC/USD
3  2022-04-06  43179.40  BTC/USD
4  2022-04-05  45479.20  BTC/USD
..        ...       ...      ...
28 2022-03-12    290.79  BCH/USD
29 2022-03-11    287.61  BCH/USD
30 2022-03-10    289.90  BCH/USD
31 2022-03-09    307.56  BCH/USD
32 2022-03-08    284.80  BCH/USD

[594 rows x 3 columns]


In [110]:
# Random Crypto
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame(tickers, columns = ['ticker'])
    for n in range(0, n):
        random_portfolios[f'Random Portfolio {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'Random Portfolio {n+1}'] = random_portfolios[f'Random Portfolio {n+1}'] / sum(random_portfolios[f'Random Portfolio {n+1}'])
    return(random_portfolios)



random_crypto = random_portfolios_generator(crypto_tickers, 1)
random_crypto

,ticker,Random Portfolio 1
0,BTC/USD,0.100883
1,ETH/USD,0.088925
2,LUNA/USD,0.007806
3,SOL/USD,0.044994
4,XRP/USD,0.052472
5,ADA/USD,0.055275
6,AVAX/USD,0.091389
7,DOT/USD,0.074830
8,DOGE/USD,0.033713
9,SHIB/USD,0.011099


In [82]:

# kraken = ccxt.kraken({
#     'apiKey': KRAKEN_API_KEY_ID,
#     'secret': KRAKEN_API_SECRET_KEY,
# })

# markets = kraken.load_markets()

# print(kraken)
# if (kraken.has['fetchTickers']):
#     print(kraken.fetch_tickers(['BTC/USD','ETH/USD','LUNA/USD','SOL/USD','XRP/USD','ADA/USD',
#     'AVAX/USD','DOT/USD','DOGE/USD','SHIB/USD','MATIC/USD',
#     'DAI/USD','LTC/USD','ATOM/USD','LINK/USD','UNI/USD','TRX/USD','BCH/USD'])) 

SyntaxError: unexpected EOF while parsing (3957576514.py, line 12)

In [10]:
#Alpaca 

alpaca_api = REST()

#Facebook, Amazon, Apple, Netflix, Google, Microsoft, Uber, Lyft, Airbnb

FANNGMULA_tickers = ["META", "AMZN", "AAPL", "NTFX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]

alpaca_api.get_bars(FANNGMULA_tickers, TimeFrame(45, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-06-08 07:30:00+00:00,126.10,126.16,125.9600,126.06,20951,304,126.049447,AAPL
2021-06-08 08:15:00+00:00,126.05,126.30,126.0500,126.30,21181,349,126.231904,AAPL
2021-06-08 09:00:00+00:00,126.27,126.32,126.2200,126.28,15955,308,126.284120,AAPL
2021-06-08 09:45:00+00:00,126.29,126.40,125.9000,125.90,30179,582,126.196877,AAPL
2021-06-08 10:30:00+00:00,125.90,126.75,125.8400,126.75,105380,1376,126.530863,AAPL
...,...,...,...,...,...,...,...,...
2021-06-08 20:15:00+00:00,49.81,49.88,49.7800,49.86,21235,97,49.860162,UBER
2021-06-08 21:00:00+00:00,49.86,49.88,49.8600,49.86,44362,30,49.879444,UBER
2021-06-08 21:45:00+00:00,49.86,49.88,49.8000,49.85,1739,12,49.864410,UBER
